## Convert Audio to image

In [2]:
import librosa.display
import librosa
import soundfile as sf
from pydub import AudioSegment

import matplotlib.pyplot as plt

import pandas as pd

import numpy as np
from IPython.display import Audio

# from tensorflow import keras

import csv
import random
import os
import shutil

Create a dataset from the urban sound 8K dataset that contains all engine idling sounds as well as an equal number of random sounds taken from the other categories. Doing so using the UrbanSound8k csv with the labels and IDs of the files


In [3]:
#Giacomos paths
#csv
us_file = '/Users/giacomo/Documents/lavoro/sound8k_data/UrbanSound8K.csv'
#the keggle dataset
sound_folder = '/Users/giacomo/Documents/lavoro/sound8k_data/keggle_dataset/'

path_to_wav = '/Users/giacomo/Documents/lavoro/sound8k_data/our_dataset/'
path_to_wav_decoded = '/Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/'

path_to_png = '/Users/giacomo/Documents/lavoro/sound8k_data/mel_specs/'

In [3]:
#Matthiass paths
#csv
#us_file = 'C:/Users/matthias/Documents/Projects/urban_sound_files/UrbanSound8K.csv'
#the keggle dataset


In [4]:
# create a list with all the IDs for car engine idling sounds
list_ideng_files = []
list_of_other_sounds = []

#set a variable that can be used later. Out of the 8732 files 1000 are idling engine sounds. 
# 12.9% are the amount of files (1000) that are not idling_engine sounds The variable is needed to pick files that are non idling_engine sounds
# with a random probability of 12.9 % in order to create a balanced dataset containing idling_engine and non idling_engine sounds.  
append_probability = 12.93 # 100 / 7732 * 100
random.seed(50)


with open(us_file) as csv_file:
    reader = csv.reader(csv_file)
    
    for line in reader:
        if 'engine_idling' in line:
            list_ideng_files.append(line[0])
        else: 
            # append lines with a probability of 12.9 %
            if random.randint(0,100) <= append_probability: 
                    list_of_other_sounds.append(line[0])

print('you have ', len(list_ideng_files), 'idling engine files \n' 'and ', len(list_of_other_sounds), 'non engine sounds')

you have  1000 idling engine files 
and  1015 non engine sounds


In [ ]:
    
# copy all files that are found in either one of the two lists.

# create a list of all files paths from the dataset within the subfolders 

sound_files =[os.path.join(root, file) for root, directories, files in os.walk(sound_folder) for file in files]

print(sound_files)


In [ ]:
# Now copy all files present in the two lists into a new folder (destination folder var)
for path in sound_files:
    filename = os.path.basename(path)
    if filename in (list_ideng_files + list_of_other_sounds):
        destination = os.path.join(path_to_wav, filename)
        shutil.copy2(path, path_to_wav)

## Standardize imput 


Some files are recorded with one, some with two channels. Lets convert all files to two channels.
Also, some have a sampling rate of 48.000 Hz, some of 44.100 Hz. Here we need to make all imput arrays the same size.
Last we make all files the same length by adding silence to shorter files

In [ ]:
import os
import subprocess

# Directory containing the files to convert
input_dir = path_to_wav

# Directory to save the converted files
output_dir = path_to_wav_decoded

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop over all files in the input directory
for filename in os.listdir(input_dir):
    # Skip non-WAV files
    if not filename.endswith('.wav'):
        continue

    # Full path to the original file
    input_file = os.path.join(input_dir, filename)

    # Full path to the converted file
    output_file = os.path.join(output_dir, filename)

    # Convert the file with ffmpeg
    subprocess.run(['ffmpeg','-y', '-i', input_file, '-c:a', 'pcm_s16le', output_file])

In [ ]:
# get full path of the audio files
full_file_paths = [os.path.join(path_to_wav_decoded, filename) 
                   for filename in os.listdir(path_to_wav) 
                   if os.path.isfile(os.path.join(path_to_wav, filename)) and filename.endswith('.wav')] 


def standardize_audio(audio, target_length):
    # apply sampling rate of 44100 and mono to all files
    _ = librosa.load(audio, sr=44100, mono = True) 
    
    # Add silence to shorter files
    current_audio = AudioSegment.from_wav(audio)
    current_length = len(current_audio)
    
    if current_length < target_length: 
        # get difference in duration and calculate the sound of silence
        silence_duration = target_length - current_length
        # create a sound of silence with the needed length   
        silence = AudioSegment.silent(duration=silence_duration)
        #add the sound of silence at the end of the file
        padded_audio = current_audio + silence
        return padded_audio
    else:
        padded_audio = current_audio
    
    padded_audio.export(audio, format="wav")
        
# find max duration in ms within all audio files      
from pydub import AudioSegment

# List of audio file extensions
audio_extensions = ['wav']

# Get length of audio files, skipping non-audio files
target_length = max([len(AudioSegment.from_file(file)) for file in full_file_paths])

# loop trough all files and apply the standardization
# To save some space on the disc we overwrite the files instead of creating new ones!
for file in full_file_paths:
    standardize_audio(file, target_length) 


In [20]:
# data augmentation

from multiprocessing import Pool
from intersection_effectivity_functions import process_file
from functools import partial

full_file_paths = [os.path.join(path_to_wav_decoded, filename) 
                   for filename in os.listdir(path_to_wav) 
                   if os.path.isfile(os.path.join(path_to_wav, filename)) and filename.endswith('.wav')] 

# Create a new function with 'output_dir' pre-filled
process_file_with_output_dir = partial(process_file, output_dir=path_to_wav_decoded)

# Create a pool of workers and apply 'process_file' function to each file
with Pool() as pool:
    pool.map(process_file_with_output_dir, full_file_paths)

saved new file as  /Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/29721-4-0-14_aug.wav
saved new file as  /Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/203929-7-5-1_aug.wav
saved new file as  /Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/7965-3-16-0_aug.wav
saved new file as  /Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/158597-2-0-85_aug.wav
saved new file as  /Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/99180-9-0-0_aug.wav
saved new file as  /Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/73524-0-0-127_aug.wav
saved new file as  /Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/44737-5-0-2_aug.wav
saved new file as  /Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/188824-7-11-0_aug.wav
saved new file as  /Users/giacomo/Documents/lavoro/sound8k_data/our_dataset_decoded/94632-5-1-26_aug.wav
saved new file as  /Users/giacomo/Documents/lavoro/soun

## Create mel spectrograms

First we check one of the samples just to make sure everything is right. We take a look at a waveplot as well as a mel spectrogram.
After that we will create mel spectrograms for each audio file.

In [ ]:
# select an audio file as an example
audio_file =  path_to_wav_decoded  + '2937-1-0-0.wav'

#If samples are not already, convert to np.array, can probably be simpflified, no need for if check

y, sr = librosa.load(audio_file, sr=44100, mono = True)
if not isinstance(y, np.ndarray):
    samples = np.array(y)


plt.figure(figsize=(14, 5))
librosa.display.waveshow(y, sr=sr, color='b') #color specification needed due to version incompatibility problems btw librosa and matplotlib
plt.title('Waveform Plot')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.tight_layout()
plt.tight_layout()
plt.show()

In [ ]:
# Listen to the audio 
Audio(audio_file)


In [10]:
#Apply fourier transformation and plot 
sgram = librosa.stft(y)
librosa.display.specshow(np.abs(sgram), sr=sr, x_axis='time', y_axis='linear')

NameError: name 'y' is not defined

In [9]:
# Create a mel spectrogramm 
sgram_mag, _ = librosa.magphase(sgram)
mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sr)
librosa.display.specshow(mel_scale_sgram)


NameError: name 'sgram' is not defined

In [ ]:

mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)
librosa.display.specshow(mel_sgram, sr=sr, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')

In [11]:
def create_mel_specs (path_to_wav, path_to_png):

    #create a list of all wav files needed to be converted: 
    
    file_list = [file for file in os.listdir(path_to_wav) if file.endswith('.wav')]
    #print(file_list)
    
    for file in file_list:    
        print(file)
        #create correct file name for saving
        audio_file = os.path.join(path_to_wav, file)
        output_file = os.path.join(path_to_png, os.path.splitext(file)[0] + '.png')
        
        # load the files and transfrom into np array
        samples, sample_rate = librosa.load(audio_file, sr=44100)
        if not isinstance(samples, np.ndarray):
            samples = np.array(samples)
        
        #apply a stft transformation    
        stft = librosa.stft(samples)

        #modify it in order to use the mel-scale instead of frenquency
        mel_scale_spec, par = librosa.magphase(stft)
        mel_spec = librosa.feature.melspectrogram(S=mel_scale_spec, sr=sample_rate)

        # Go from amplitude to db 
        mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.min)

        # create and save the figure
        plt.figure(figsize=(293/100, 293/100), dpi=100) # size in inches        
        librosa.display.specshow(mel_spec_db, x_axis='time', y_axis='mel', sr=sample_rate)
        plt.axis('off')

        #save plot without any white border
        plt.savefig(output_file, bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close()

In [21]:
#Run the function to create the mel specs

test = create_mel_specs(path_to_wav_decoded, path_to_png)

106014-5-0-2_aug.wav
17480-2-0-6_aug.wav
203929-7-5-1.wav
187378-3-0-7_aug.wav
185375-9-0-60_aug.wav
7965-3-16-0.wav
44737-5-0-2.wav
201652-5-0-2_aug.wav
209992-5-2-81.wav
113202-5-0-9_aug.wav
94632-5-0-0.wav
155217-9-1-24.wav
50668-5-3-0_aug.wav
168846-5-1-6_aug.wav
174873-3-5-1_aug.wav
17853-5-0-11.wav
177592-5-0-3_aug.wav
201988-5-0-8.wav
29721-4-0-14_aug.wav
195451-5-0-4.wav
80806-2-0-3.wav
107190-3-0-0_aug.wav
144068-5-0-18_aug.wav
103074-7-3-1_aug.wav
201988-5-0-21.wav
164797-2-0-44_aug.wav
154758-5-0-8_aug.wav
128160-5-0-15.wav
209992-5-2-42.wav
123399-2-0-83_aug.wav
168713-9-0-32_aug.wav
188823-7-2-0.wav
118558-5-1-2_aug.wav
174276-7-5-0.wav
34771-3-0-5.wav
132016-7-0-8_aug.wav
22883-7-51-0_aug.wav
111386-5-1-13_aug.wav
91209-5-0-1_aug.wav
39856-5-0-28.wav
30204-0-0-7_aug.wav
58857-2-0-14.wav
55728-9-0-67_aug.wav
15544-5-0-17_aug.wav
62567-5-0-0_aug.wav
195451-5-0-11.wav
156418-3-0-0_aug.wav
102857-5-0-9_aug.wav
134717-0-0-28_aug.wav
39856-5-0-14.wav
61503-2-0-5.wav
93567-8-0-3

## Create NNs

## import metadata 

import the UrbanSound8k csv, filter out all entries that are not used in this project and take a look at the data.


In [5]:
#import csv with labels
df = pd.read_csv(us_file, header=0)

filtered_df = df[df['slice_file_name'].isin(list_ideng_files + list_of_other_sounds)]
#print(filtered_df.head(10))
filtered_df.loc[filtered_df['class'] != 'engine_idling', 'class'] = 'non_idling_engine'
print(filtered_df.head(5))

# We now want to add the augmented files to this dataframe in order to attach the correct label to them
def add_augmentation(row):
    row['slice_file_name'] = row['slice_file_name'].replace('.wav', '_aug.wav')
    return row

# Apply the function and concatenate the results
df_augmented = pd.concat([filtered_df, filtered_df.apply(add_augmentation, axis=1)], ignore_index=True)
print(df_augmented['class'].value_counts())

      slice_file_name    fsID  start   end  salience  fold  classID  \
8   100263-2-0-36.wav  100263   18.0  22.0         1     5        2   
14   100652-3-0-0.wav  100652    0.0   4.0         1     2        3   
17   100652-3-0-3.wav  100652    1.5   5.5         1     2        3   
22   100852-0-0-0.wav  100852    0.0   4.0         1     5        0   
28  100852-0-0-14.wav  100852    7.0  11.0         1     5        0   

                class  
8   non_idling_engine  
14  non_idling_engine  
17  non_idling_engine  
22  non_idling_engine  
28  non_idling_engine  
class
non_idling_engine    2030
engine_idling        2000
Name: count, dtype: int64


In [6]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Preprocess and batch the dataset (add any necessary preprocessing here)
data_df = df_augmented[['slice_file_name', 'class']]
data_df.loc[:,'slice_file_name'] = data_df['slice_file_name'].str.replace('.wav', '.png')
dataset_path = '/Users/giacomo/Documents/lavoro/sound8k_data/mel_specs/' #where the mel specs are


# Split the data into train+validation and test sets
train_val_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42)

# Further split the train+validation set into separate train and validation sets
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Create an ImageDataGenerator for data loading and preprocessing
datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to the range [0, 1]
)

# Use flow_from_dataframe to load and preprocess the images
batch_size = 32  # Adjust this according to your needs

# Train generator
train_generator = datagen.flow_from_dataframe(
    train_df,
    directory=dataset_path,
    x_col="slice_file_name",  # Column containing filenames
    y_col="class",  # Column containing class labels
    target_size=(224, 224),  # Reshape your images to a desired size
    batch_size=batch_size,
    class_mode='categorical',  # If you have multiple classes
    validate_filenames=True  # For the training set
)

# Validation generator
validation_generator = datagen.flow_from_dataframe(
    val_df,
    directory=dataset_path,
    x_col="slice_file_name",
    y_col="class",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    validate_filenames=True # For the validation set
)

# Test generator
test_generator = datagen.flow_from_dataframe(
    test_df,
    directory=dataset_path,
    x_col="slice_file_name",
    y_col="class",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    validate_filenames=True # For the test set
)

num_classes = len(train_generator.class_indices)  # Get the number of classes

Found 2418 validated image filenames belonging to 2 classes.
Found 806 validated image filenames belonging to 2 classes.
Found 806 validated image filenames belonging to 2 classes.


In [16]:
#test different models architectire and hyperparameters values
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from kerastuner.tuners import RandomSearch

# rest of your code

def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int('conv_1_units', min_value=32, max_value=256, step=32), (3, 3), activation='leaky_relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Conv2D(hp.Int(f'conv_{i+2}_units', min_value=32, max_value=256, step=32), (3, 3), activation='leaky_relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(hp.Float(f'dropout_{i+2}', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Flatten())
    model.add(Dense(hp.Int('dense_units', min_value=32, max_value=256, step=32), activation='leaky_relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='engine_idling')

tuner.search_space_summary()

tuner.search(train_generator, 
             epochs=5, 
             validation_data=validation_generator)

tuner.results_summary()

Using TensorFlow backend


/var/folders/65/jjhht_kn6479d_8drvxjcgjh0000gn/T/ipykernel_63564/4072901755.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Search space summary
Default search space size: 6
conv_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
conv_2_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
dropout_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
192               |192               |conv_1_units
0.3               |0.3               |dropout_1
3                 |3              

KeyboardInterrupt: 

In [8]:
#last model experiment with this (sunday)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='leaky_relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0))

    model.add(Conv2D(64, (3, 3), activation='leaky_relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3, 3), activation='leaky_relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(256, (3, 3), activation='leaky_relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (3, 3), activation='leaky_relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Flatten())
    
    model.add(Dense(512, activation='leaky_relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(256, activation='leaky_relu'))
    model.add(Dropout(0.2))  # Add dropout after the first dense layer
    

    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='Adam')
    return model

model = build_model()


from datetime import datetime
import pickle
# Get the current time
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")

# Save the model architecture
model_json = model.to_json()
with open(f"model_{timestamp}.json", "w") as json_file:
    json_file.write(model_json)

# Save the learning rate
with open(f"learning_rate_{timestamp}.txt", "w") as lr_file:
    lr_file.write(str(0.01))  # Replace with your learning rate

# Train the model
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)

# Save the training results
with open(f'history_{timestamp}.pickle', 'wb') as history_file:
    pickle.dump(history.history, history_file)

Epoch 1/20
76/76 [==============================] - 62s 810ms/step - loss: 0.7361 - accuracy: 0.4946 - val_loss: 0.6967 - val_accuracy: 0.4839
Epoch 2/20
76/76 [==============================] - 67s 883ms/step - loss: 0.6976 - accuracy: 0.5343 - val_loss: 0.6792 - val_accuracy: 0.5757
Epoch 3/20
76/76 [==============================] - 70s 922ms/step - loss: 0.6537 - accuracy: 0.5864 - val_loss: 0.6634 - val_accuracy: 0.6278
Epoch 4/20
76/76 [==============================] - 70s 925ms/step - loss: 0.6388 - accuracy: 0.6108 - val_loss: 0.6052 - val_accuracy: 0.6650
Epoch 5/20
76/76 [==============================] - 70s 924ms/step - loss: 0.4830 - accuracy: 0.7461 - val_loss: 0.4055 - val_accuracy: 0.8176
Epoch 6/20
76/76 [==============================] - 71s 936ms/step - loss: 0.4126 - accuracy: 0.8069 - val_loss: 0.3533 - val_accuracy: 0.8226
Epoch 7/20
76/76 [==============================] - 72s 950ms/step - loss: 0.3681 - accuracy: 0.8391 - val_loss: 0.3750 - val_accuracy: 0.8114

In [ ]:
#latest version
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='leaky_relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0))

    model.add(Conv2D(64, (3, 3), activation='leaky_relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3, 3), activation='leaky_relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(256, (3, 3), activation='leaky_relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (3, 3), activation='leaky_relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Flatten())
    
    model.add(Dense(512, activation='leaky_relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(256, activation='leaky_relu'))
    model.add(Dropout(0.2))  # Add dropout after the first dense layer
    

    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='Adam')
    return model

model = build_model()


from datetime import datetime
import pickle
# Get the current time
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")

# Save the model architecture
model_json = model.to_json()
with open(f"model_{timestamp}.json", "w") as json_file:
    json_file.write(model_json)

# Save the learning rate
with open(f"learning_rate_{timestamp}.txt", "w") as lr_file:
    lr_file.write(str(0.01))  # Replace with your learning rate

# Train the model
history = model.fit(train_generator, epochs=30, validation_data=validation_generator)

# Save the training results
with open(f'history_{timestamp}.pickle', 'wb') as history_file:
    pickle.dump(history.history, history_file)

In [ ]:
# launch tensorboard
%load_ext tensorboard
%tensorboard --logdir logs/fit
